
2. 콘텐츠 기반 컨텐츠(Content Based Filtering)
1) 켄텐츠에 포함되는 텍스트들을 (장르, 키워드, 감독 등) 선별한다.
2) 정해진 열에 텍스트들을 전부 집어넣고 string 형태로 변환 (ex : "호러 개그 액션")
3) 2)에서 구한 string에서 언급된 단어별로 count함.
4) 각 단어별로 count한 것을 통해 백터화를 할 수가 있음
5) 이제 각 벡터간의 거리를 측정함
6) 벡터간의 거리가 적을수록 비슷한 콘텐츠임

올인원에듀테크에서 포함시켜볼 텍스트들 : 장르, 성별, 


In [1]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
movies = pd.read_csv('tmdb_5000_movies.csv')
#movies2 = pd.read_csv('keywords.csv', encoding='utf-8')

#movies['id']=movies['id'].astype(int)
#movies2['id']=movies2['id'].astype(int)
#movies = pd.merge(movies, movies2,on='id')
movies_df = movies[['id','title', 'genres','keywords', 'vote_average', 'vote_count',
                 'popularity', 'overview', 'production_companies']]

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
movies_df['production_companies'] = movies_df['production_companies'].apply(literal_eval)
# genres, keywords에서 id를 제외하고 name만 불러옴.
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: [y['name'] for y in x])

In [2]:
def connect(input):
    for i in range(len(input)):
        input[i] = input[i].replace(' ','_')
    return input

def cut_2_more(input, n = 2):
    #print(input)
    if(len(input) > n):
        input = input[:n]
    return input

def sum_pro_genre(input):
    return ' '.join(input['production_companies']) + ' ' + ' '.join(input['genres'])
movies_df['genres'] = movies_df['genres'].apply(connect)
movies_df['keywords'] = movies_df['keywords'].apply(connect)
movies_df['production_companies'] = movies_df['production_companies'].apply(connect)
#movies_df['genres'] = movies_df['genres'].apply(cut_2_more)
#movies_df['keywords'] = movies_df['keywords'].apply(cut_2_more)
movies_df['production_companies'] = movies_df['production_companies'].apply(cut_2_more)

print(movies_df['production_companies'][0])
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x))
movies_df['production_companies_literal'] = movies_df.apply(sum_pro_genre, axis = 1)

['Ingenious_Film_Partners', 'Twentieth_Century_Fox_Film_Corporation']


In [3]:
movies_df['production_companies_literal']

0       Ingenious_Film_Partners Twentieth_Century_Fox_...
1       Walt_Disney_Pictures Jerry_Bruckheimer_Films A...
2         Columbia_Pictures Danjaq Action Adventure Crime
3       Legendary_Pictures Warner_Bros. Action Crime D...
4       Walt_Disney_Pictures Action Adventure Science_...
                              ...                        
4798              Columbia_Pictures Action Crime Thriller
4799                                       Comedy Romance
4800    Front_Street_Pictures Muse_Entertainment_Enter...
4801                                                     
4802    rusty_bear_entertainment lucky_crow_films Docu...
Name: production_companies_literal, Length: 4803, dtype: object

In [5]:
# 모든 변수가 텍스트로 이루어졌으니 Countvectorizer로 feature 벡터화시키기
# 경우에 따라 추가적으로 Lemmatization 추가해 토큰화 전용 함수 추가도 가능!

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
keywords_vect = cnt_vect.fit_transform(movies_df['keywords_literal'])
tfid_vect = TfidfVectorizer().fit_transform(movies_df['keywords_literal'])
genres_vect = TfidfVectorizer().fit_transform(movies_df['genres_literal'])
production_companies_vect = TfidfVectorizer().fit_transform(movies_df['production_companies_literal'])
#genres_tfid_vect

# 장르에 따른 영화별 코사인 유사도 추출
genre_sim = cosine_similarity(genres_vect, genres_vect)
keywords_sim = cosine_similarity(keywords_vect,keywords_vect)
production_companies_sim = cosine_similarity(production_companies_vect,production_companies_vect)
#tfid_sim = cosine_similarity(tfid_vect,tfid_vect)
# 3개만 유사도행렬값 추출해보기
print(genre_sim[:3])
print(genre_sim[0][3133])
print(type(genre_sim))

[[1.         0.8450897  0.48696815 ... 0.         0.         0.        ]
 [0.8450897  1.         0.5762325  ... 0.         0.         0.        ]
 [0.48696815 0.5762325  1.         ... 0.         0.         0.        ]]
0.47676936989909646
<class 'numpy.ndarray'>


In [7]:

def sort_similar(input,output):
    for i in range(len(input)):
        output[i] = input[i].argsort()[::-1]
production_companies_sim_idx = production_companies_sim.argsort()
keywords_sim_idx = keywords_sim.argsort()
genre_sim_idx = genre_sim.argsort()
sort_similar(production_companies_sim,production_companies_sim_idx)
sort_similar(genre_sim,genre_sim_idx)
sort_similar(keywords_sim,keywords_sim_idx)
print(type(genre_sim_idx))
print(genre_sim[0][232])
np.save("genre",genre_sim_idx)


<class 'numpy.ndarray'>
1.0


In [21]:
# argsort를 이용해서 유사도가 높은 영화들의 index 추출
#tfid_sim_idx = tfid_sim.argsort()[::-1]
#sum_idx = (genre_sim + tfid_sim_idx).argsort()[::-1]
#print(tfid_sim==keywords_sim_idx)
# 특정 영화를 기준으로 선정해서 유사도가 높은 영화를 반환하는 함수 생성
def find_sim_movie(df, sorted_idx, title_name, top_n=10):
    # 비교기준으로 선정할 특정 영화 선정
    title_movie = df[df['title'] == title_name]
    # 비교기준 영화의 index.values를 할당해 유사도 행렬에서 비교기준 영화에 해당하는 유사도 행렬값을 찾자!
    title_movie_idx = title_movie.index.values[0]
    # 모든 영화끼리의 유사도 행렬에서 비교기준 영화에 대한 유사도행렬 찾아서 할당
    sim_idx = sorted_idx[title_movie_idx]
    top_sim_idx = np.array([])
    pos = 0
    while(len(top_sim_idx) < len(sim_idx) and len(top_sim_idx) < top_n):
        if (sim_idx[pos] != title_movie_idx):
            top_sim_idx = np.append(top_sim_idx,sim_idx[pos])
        pos += 1
    
    # top_sim_idx는 2차원 array이기 때문에 1차원 array로 변경
    #similar_movie = df.iloc[top_sim_idx]
    similar_movie = df.iloc[top_sim_idx].sort_values('vote_average', ascending=False)
    
    return similar_movie

movie_name = 'Harry Potter and the Half-Blood Prince'
print(movies_df[movies_df['title'] == movie_name]['genres'])
print(movies_df[movies_df['title'] == movie_name]['keywords'])
#print(movies_df[movies_df['title'] == movie_name]['keywords'])
similar_movies = find_sim_movie(movies_df, genre_sim_idx, movie_name)
similar_movies_keyword = find_sim_movie(movies_df, keywords_sim_idx, movie_name)
similar_movies_production = find_sim_movie(movies_df, production_companies_sim_idx, movie_name)
#similar_movies_sum = find_sim_movie(movies_df, sum_idx, movie_name)
#similar_movies_tfid = find_sim_movie(movies_df, tfid_sim_idx, movie_name)
print(similar_movies[['title','genres']])
print(similar_movies_keyword[['title','keywords']])
print(similar_movies_production[['title','production_companies_literal']])
#print(similar_movies_tfid[['title','genres']])

8    [Adventure, Fantasy, Family]
Name: genres, dtype: object
8    [witch, magic, broom, school_of_witchcraft, wi...
Name: keywords, dtype: object
                                                  title  \
114                 Harry Potter and the Goblet of Fire   
1764                                       Return to Oz   
32                                  Alice in Wonderland   
15             The Chronicles of Narnia: Prince Caspian   
109   The Chronicles of Narnia: The Voyage of the Da...   
1322                                      City of Ember   
175                                             The BFG   
781                                            Inkheart   
341   Percy Jackson & the Olympians: The Lightning T...   
1999         The Adventurer: The Curse of the Midas Box   

                            genres  
114   [Adventure, Fantasy, Family]  
1764  [Adventure, Family, Fantasy]  
32    [Family, Fantasy, Adventure]  
15    [Adventure, Family, Fantasy]  
109   [Adventure, 